In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action="ignore")
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split, StratifiedKFold
import torch
from torch.utils.data import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
data_doc = pd.read_csv("/content/drive/MyDrive/Microsft LMR/Preprocessed_data_doc.csv",index_col=0)
test_doc = pd.read_csv("/content/drive/MyDrive/Microsft LMR/Preprocessed_test_doc.csv", index_col=0)

In [27]:
data_doc.head()

,tweet_id,text,location,cleaned_text
1,ID_1001136696589631488,"Flash floods struck a Maryland city on Sunday,...",Maryland,flash flood strike maryland city sunday wash s...
2,ID_1001136950345109504,State of emergency declared for Maryland flood...,Maryland,state emergency declare maryland flooding
3,ID_1001137334056833024,Other parts of Maryland also saw significant d...,Baltimore Maryland,maryland significant damage sunday storm inclu...
4,ID_1001138374923579392,"Catastrophic Flooding Slams Ellicott City, Mar...",Ellicott City Maryland,catastrophic flooding slam ellicott city maryl...
5,ID_1001138377717157888,WATCH: 1 missing after flash #FLOODING devasta...,Ellicott City Maryland,watch miss flash flooding devastate ellicott c...


In [28]:
data_doc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16448 entries, 1 to 73071
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tweet_id      16448 non-null  object
 1   text          16448 non-null  object
 2   location      16448 non-null  object
 3   cleaned_text  16448 non-null  object
dtypes: object(4)
memory usage: 642.5+ KB


In [8]:
data_doc['text'][3],data_doc['location'][43],data_doc['cleaned_text'][3]

('Other parts of Maryland also saw significant damage from Sundays storms including this Baltimore city neighborhood, #Dundalk and #Catonsville. Rain totals spanned from 1 to 10 inches across Maryland:  #ECFlood',
 'EllicottCity Maryland',
 'maryland significant damage sunday storm include baltimore city neighborhood dundalk catonsville rain total span inch maryland ecflood')

In [29]:
train_doc = data_doc.drop(['tweet_id','text'], axis=1)
Test_doc = test_doc.drop(['tweet_id','text'], axis=1)
train_doc.head()

,location,cleaned_text
1,Maryland,flash flood strike maryland city sunday wash s...
2,Maryland,state emergency declare maryland flooding
3,Baltimore Maryland,maryland significant damage sunday storm inclu...
4,Ellicott City Maryland,catastrophic flooding slam ellicott city maryl...
5,Ellicott City Maryland,watch miss flash flooding devastate ellicott c...


In [ ]:
threshold = 10

# Count occurrences of each unique value in the 'location' column
location_counts = train_doc['location'].value_counts()

# Filter out entries with less than threshold occurrences
Train_doc = train_doc[train_doc['location'].isin(location_counts[location_counts >= threshold].index)]

In [30]:
location_encoder = LabelEncoder()
enc_location = location_encoder.fit_transform(Train_doc['location'])
enc_location

array([1997, 1997,  362, ..., 2042, 2042, 2042])

In [31]:
#This code creates a dictionary that maps numerical class indices
#to their corresponding class labels from the encoder.
#It pairs each index with its respective label for easy lookup.
class_indices = {val:key for key,val in zip(location_encoder.classes_, range(len(location_encoder.classes_)))}

In [32]:
Train_doc['enc_location'] = enc_location

In [13]:
#discriminator = ElectraForPreTraining.from_pretrained("google/electra-large-discriminator")
#tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-large-discriminator")

config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForPreTraining: ['electra.embeddings_project.bias', 'electra.embeddings_project.weight']
- This IS expected if you are initializing ElectraForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [33]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
tokenizer

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [34]:
class MSL_DataSet(Dataset):
    def __init__(self, X, y, tokenizer):
        self.X = X.to_list()
        self.y = y.to_list()

        print("Performing text tokenization on X")

        self.tokenized_X = tokenizer(self.X, truncation=True, padding=True, return_tensors="pt")

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        data = {key: val[index] for key, val in self.tokenized_X.items()}
        data["labels"] = torch.tensor(self.y[index], dtype=torch.long)
        return data



In [ ]:
# Split the dataset into train and validation sets with stratification
X_train, X_val, y_train, y_val = train_test_split(
    Train_doc["cleaned_text"],
    Train_doc["enc_location"],
    test_size=0.2,
    stratify=Train_doc["enc_location"]
)

In [ ]:
# Create train and validation datasets
train_data = MSL_DataSet(X=X_train, y=y_train, tokenizer=tokenizer)
val_data = MSL_DataSet(X=X_val, y=y_val, tokenizer=tokenizer)


In [ ]:
# Define the compute_metrics function
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = metrics.accuracy_score(y_true=labels, y_pred=pred)
    recall = metrics.recall_score(y_true=labels, y_pred=pred, average='weighted') # AVERAGE ==  WEIGHTED CAUSE ITS NOT A BINARY CLASSIFICATION
    precision = metrics.precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = metrics.f1_score(y_true=labels, y_pred=pred, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [35]:
#train_data = MSL_DataSet(X=Train_doc["cleaned_text"], y=Train_doc["enc_location"], tokenizer=tokenizer)
#val_data = MSL_DataSet(X=Train_doc["cleaned_text"], y=Train_doc["enc_location"], tokenizer=tokenizer)

Performing text tokenization on X
Performing text tokenization on X


In [36]:
#def compute_metrics(p):
#    pred, labels = p
#    pred = np.argmax(pred, axis=1)

 #   accuracy = metrics.accuracy_score(y_true=labels, y_pred=pred)
 #   recall = metrics.recall_score(y_true=labels, y_pred=pred, average='weighted') # AVERAGE ==  WEIGHTED CAUSE ITS NOT A BINARY CLASSIFICATION
 #   precision = metrics.precision_score(y_true=labels, y_pred=pred, average='weighted')
  #  f1 = metrics.f1_score(y_true=labels, y_pred=pred, average='weighted')

  #  return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [37]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)

cuda


In [38]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(class_indices))

model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [39]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [40]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=20,             # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,                # log metrics every 10 steps
    evaluation_strategy="epoch",     # evaluate every logging step
)

trainer = Trainer(
    model=model,                      # the instantiated 🤗 Transformers model to be trained
    args=training_args,               # training arguments, defined above
    train_dataset=train_data,      # training dataset
    eval_dataset=val_data,         # evaluation dataset
    compute_metrics=compute_metrics   # function to compute metrics
)

In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.399100,3.080089,0.598918,0.383553,0.598918,0.463554
2,3.387800,2.493167,0.632964,0.432601,0.632964,0.508300
3,2.057900,2.058201,0.661114,0.488681,0.661114,0.547646
4,1.669400,1.684335,0.691573,0.527140,0.691573,0.592300
5,1.707200,1.397613,0.713461,0.566983,0.713461,0.624764
6,1.197400,1.116596,0.755411,0.641176,0.755411,0.684975
7,1.004300,0.884062,0.794747,0.702742,0.794747,0.736984
8,0.806100,0.676331,0.839312,0.770904,0.839312,0.795214
9,0.964900,0.501850,0.890625,0.841735,0.890625,0.858897
10,0.603200,0.360760,0.929232,0.895056,0.929232,0.907266


TrainOutput(global_step=20560, training_loss=1.0161568481040255, metrics={'train_runtime': 3576.643, 'train_samples_per_second': 91.975, 'train_steps_per_second': 5.748, 'total_flos': 6624677750883840.0, 'train_loss': 1.0161568481040255, 'epoch': 20.0})

In [42]:
trainer.evaluate()

{'eval_loss': 0.02319571003317833,
 'eval_accuracy': 0.9996352140077821,
 'eval_precision': 0.9993788505188067,
 'eval_recall': 0.9996352140077821,
 'eval_f1': 0.9994855973576792,
 'eval_runtime': 37.4283,
 'eval_samples_per_second': 439.454,
 'eval_steps_per_second': 6.866,
 'epoch': 20.0}

In [43]:
trainer.save_model('MLLM_model')

In [44]:
Test_doc

,cleaned_text
0,happen infrastructure new england global warm ...
1,solder miss flood pray eddison hermond pray el...
2,rt police search miss person devastating year ...
3,flash flood tear maryland town second time yea...
4,ellicott city flooding picture maryland govern...
...,...
2937,want donate food homeless cat affect earthquak...
2938,I live mexico earthquake house damage moment I...
2939,rt watch national taco day calgary benefit mex...
2940,oaxaca chiapas mexico poor state harsh blow sc...


In [45]:
trained_model = DistilBertForSequenceClassification.from_pretrained("MLLM_model")

In [46]:
def MakePrediction(text, model, tokenizer):
    tokenized_txt = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**tokenized_txt)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    return class_indices[predicted_class]

In [47]:
test_doc["location"] = test_doc["cleaned_text"].apply(lambda x: MakePrediction(x, trained_model, tokenizer))

In [48]:
submission = test_doc.drop(["text", "cleaned_text"], axis="columns")

In [51]:
submission.to_csv("/content/drive/MyDrive/Microsft LMR/distill_baseline.csv", index=False)

In [52]:
submission

,tweet_id,location
0,ID_1001154804658286592,Jamaica Haiti US
1,ID_1001155505459486720,Ellicott City Maryland
2,ID_1001155756371136512,Ellicott City Maryland
3,ID_1001159445194399744,Ellicott Maryland
4,ID_1001164907587538944,Ellicott City Maryland
...,...,...
2937,ID_915017703055749120,Mexico
2938,ID_915026957758328832,Mexico San Antonio
2939,ID_915253441726889984,Calgary Alberta
2940,ID_915971980859400192,Mexico Oaxaca
